# 1º Trabalho IA

In [113]:
import copy
import random


rows=6
cols=7

class ConnectFour:
    def __init__(self):
        self.board = [["-"] * 7 for _ in range(6)]
        self.columns_height = [5 for _ in range(7)]              # Lista para controlar a altura de cada coluna
        self.winner = False
        self.player_winner= True                                 
        self.impossible_move=False
        self.turn="X"
        self._score = 0
        self.last_move = None
        
    
    def is_full(self, column):                                   # Verifica se a coluna está cheia
        if self.columns_height[column]== -1:
            return True
        return False
    
    def is_board_full(self):
        for row in range(len(self.board)):
            if not self.is_full(row):
                return False
        return True

    
    def switch_turn(self):   
        if self.turn == "O":
            self.turn = "X"
        else:
            self.turn = "O"
    
    def move(self,column, player):
        if column < 0 or column > 6 or self.is_full(column):
            self.impossible_move=True
            return False                                         # Ou a coluna está cheia ou não existe
        
        else:
            #print(f"Altura da coluna {column} {self.columns_height[column]}")
            row = self.columns_height[column]
            self.board[row][column] = player
            self.columns_height[column] -= 1
            #print(f"Altura da coluna {column} apos o movimento {self.columns_height[column]}")
            self.impossible_move=False
            self.switch_turn()
            self.last_move = column
            return True
    
    def get_score(self,h):
        
        a=512
        b=e=50
        c=f=10
        d=g=1
        
        #print(f"O valor de h é este aqui {h}")
        
        if h=="3":                #DIFICULDADE DIFICIL
            #print(f"Dificuldade dificil")
            e=10
            f=5
            #print(h)
            
        if h=="2":                #DIFICULDADE MEDIA
            #print(f"Dificuldade media")
            a=1000
            e=10
            f=5
            d=5
            c=50
            b=100
        
        def evaluate_segment(segm):
            
            if self.player_winner == "X":
                #print("X venceu")
                return a
            
            elif self.player_winner == "O":
                #print("O venceu")
                return -a

            elif "O" in segm and "X" not in segm:
                if segm.count("O") == 3:
                    return -e
                elif segm.count("O") == 2:
                    return -f
                elif segm.count("O") == 1:
                    return -g
                
            elif segm.count("-") == 4:
                return 0
            
            elif "X" in segm and "O" not in segm:
                if segm.count("X") == 1:
                    return d
                elif segm.count("X") == 2:
                    return c
                elif segm.count("X") == 3:
                    return b            

            return 0

        #-------------------
        # Aqui começa o get_score
        #-------------------
        
        # Avaliar todos os segmentos possíveis de tamanho 4
        self._score = 0
        
        for i in range(6):     # Analisar as linhas
            for j in range(4):
                if j + 3 < 7:  # Verificar se os índices estão na matriz
                    segment = [self.board[i][j + k] for k in range(4)]
                    self._score += evaluate_segment(segment)
        
        for i in range(4):     # Analisar as colunas
            for j in range(7):
                if i + 3 < 6:  # Verificar se os índices estão na matriz
                    segment = [self.board[i + k][j] for k in range(4)]
                    self._score += evaluate_segment(segment)
                    
        for i in range(3):     # Analisar primeira diagonal
            for j in range(4):
                if i + 3 < 6 and j + 3 < 7:  # Verificar se os índices estão na matriz
                    segment = [self.board[i + k][j + k] for k in range(4)]
                    self._score += evaluate_segment(segment)
                    
        for i in range(3):     # Analisar segunda diagonal (esquerda para a direita a subir)
            for j in range(3, 7):
                if i + 3 < 6 and j - 3 >= 0:  # Verificar se os índices estão na matriz
                    segment = [self.board[i + k][j - k] for k in range(4)]
                    self._score += evaluate_segment(segment)

        return self._score 
    
    
    def check_winner(self):
        
        #check horizontal
        for i in range(rows):
            for j in range(4):
                if self.board[i][j] == self.board[i][j+1] == self.board[i][j+2] == self.board[i][j+3] != '-':
                    self.winner=True
                    self.player_winner=self.board[i][j]
                    return True
                
        #check vertical
        for i in range(3):
            for j in range(cols):
                if self.board[i][j] == self.board[i+1][j] == self.board[i+2][j] == self.board[i+3][j] != '-':
                    self.winner=True
                    self.player_winner=self.board[i][j]
                    return True
                
        #check first diagonal
        for i in range(3):
            for j in range(4):
                if self.board[i][j] == self.board[i+1][j+1] == self.board[i+2][j+2] == self.board[i+3][j+3] != '-':
                    self.winner=True
                    self.player_winner=self.board[i][j]
                    return True
        
        #check second diagonal
        for i in range(3,6):
            for j in range(4):
                if self.board[i][j] == self.board[i-1][j+1] == self.board[i-2][j+2] == self.board[i-3][j+3] != '-':
                    self.winner=True
                    self.player_winner=self.board[i][j]
                    return True
        
    
    def successors(self):
        cols=[]
        possible_successors=[]
        for i in range(0,7):
            successor=copy.deepcopy(self)
            if successor.move(i,self.turn):                   # Temos de fazer funçao para ver quem foi o ultimo a jogar
                possible_successors.append(successor)         # Adiciona a uma lista todos os movimentos possiveis
                cols.append(i)                               
                #print("-----")
                #print(f"Estado filho após o movimento {i} é ")
                #print(f"{possible_successors[i]}")            # A cruz tem de seguir as jogadas com maior pontuaçao
                #print("")
                #print("-----")
            
            else:
                possible_successors.append(None)
                cols.append(None)
                
        return possible_successors,cols

    def Astar(self, h):
        filhos, _ = self.successors()
        maximo = -float('inf')        #Valor mais baixo 
        minimo = float('inf')         #Valor mais alto
        melhor_movimento = None
        movimentos_iguais_max = []
        movimentos_iguais_min = []

        if self.turn == "X":
            for i in range(7):  
                if (filhos[i]==None):
                    continue
                filhos[i].check_winner()
                temp = filhos[i].get_score(h) + 16
                if temp > maximo:
                    maximo = temp
                    movimentos_iguais_max = [i]           #Caso seja maior a lista anterior é substituida pelo melhor movimento
                elif temp == maximo:
                    movimentos_iguais_max.append(i)       #Caso haja dois movimentos com pontuaçao igual acrescenta a lista 

            if movimentos_iguais_max:
                melhor_movimento = random.choice(movimentos_iguais_max)      #Faz random caso hajam dois movimentos 
                                                                             #ou mais com a mesma pontuaçao 
            return maximo, melhor_movimento

        elif self.turn == "O":
            for i in range(7):
                if (filhos[i]==None):
                    continue
                filhos[i].check_winner()
                temp = filhos[i].get_score(h) - 16
                if temp < minimo:
                    minimo = temp
                    movimentos_iguais_min = [i]
                elif temp == minimo:
                    movimentos_iguais_min.append(i)

            if movimentos_iguais_min:
                melhor_movimento = random.choice(movimentos_iguais_min)
            return minimo, melhor_movimento

        return None

    def minimax(self, depth,h):
        if depth == 0 or self.check_winner() or self.is_board_full():
            return self.get_score(h), None

        maximizing_player = self.turn == "X"
        best_move = None
        if maximizing_player:
            max_eval = float('-inf')
            filhos, _ = self.successors()
            for i in range(len(filhos)):
                if (filhos[i]==None):
                    continue
                #print(f"Filho: {filhos[i]}")
                temp, _ = filhos[i].minimax(depth - 1, h)
                if temp > max_eval:
                    max_eval = temp
                    best_move = i
            #print(f"Melhor movimento: {best_move}")
            return max_eval, best_move
        else:
            min_eval = float('inf')
            filhos, _ = self.successors()
            for i in range (len(filhos)):
                if (filhos[i]==None):
                    continue
                #print(f"Filho: {filhos[i]}")
                temp, _ = filhos[i].minimax(depth - 1, h)
                if temp < min_eval:
                    min_eval = temp
                    best_move = i
            #print(f"Melhor movimento: {best_move}")
            return min_eval, best_move
    
    def monte_carlo(self,root_node, simulations):
        for _ in range(simulations):
            #print(f"Root_Node: {root_node}")
            node = root_node
            #print(f"Nó: {node.game}")
            #print(f"Nó: {node.game.check_winner()}")
            if not node.expanded():
                #print(f"Entrou na expansao")
                node.expand()
            else:
                child = node.select_child()
                #print(f"Child: {child}")
                node = child
                #print(f"Nó filho: {node.game}")
                result = node.simulate()
                #print(f"Estado Final: {node}")
                #print(f"Resultado: {result}")
                node.backpropagate(result)

    
    def __str__(self):
        board_string = "\n"
        board_string += "0 1 2 3 4 5 6\n"  # Índices das colunas
        for row in self.board:
            board_string += " ".join(row) + "\n"
        return board_string


        
    

MONTE CARLO

In [114]:
from time import time
import math

class Node:
    def __init__(self,game,jogador,parent=None):
        self.game=game
        self.parent=parent
        self.children=[]
        self.wins=0
        self.visits=0
        self.jogador=jogador
    
    def expanded(self):
        return len(self.children) > 0
    
    def ucb1_value(self, child):
        if child.visits == 0:
            return float('inf')  # Retorna infinito se o filho não foi visitado para garantir que seja selecionado
        
        exploration_factor = math.sqrt(2)  # Fator de exploração (pode ser ajustado conforme necessário)
        exploitation_value = child.wins / child.visits # Fator de coiso
        exploration_value = exploration_factor * math.sqrt(math.log(self.visits) / child.visits) # Formula final
        return exploitation_value + exploration_value
    
    def select_child(self):
        if not self.children:
            #print("Nenhum filho disponível")
            return None  # Se não houver filhos, retorna None
        # Calcula o valor UCB1 para todos os filhos e escolhe o filho com maior valor UCB1
        best_child = max(self.children, key=self.ucb1_value)
        return best_child
    
    def expand(self):
        if self.game.check_winner() or self.game.is_board_full():
            #print("Jogo já terminou, não há nada para expandir")
            return  # Se o jogo já estiver terminado, não há nada para expandir

        filhos, _ = self.game.successors()  # Obtém os sucessores do estado atual do jogo

        for successor in filhos:
            #print(f"Sucessor: {successor}")
            if successor == None:
                continue
            else:
                child_node = Node(successor, self.jogador, parent=self)  # Cria um nó para cada sucessor
                self.children.append(child_node)  # Adiciona o nó como filho do nó atual

    def simulate(self):
        current_game = copy.deepcopy(self.game)
        #print(f"Check_winner: {current_game.check_winner()}")
        while not (current_game.check_winner() or current_game.is_board_full()):
            possible_moves = [col for col in range(7) if not current_game.is_full(col)]
            if possible_moves:
                random_move = random.choice(possible_moves)  # Escolhe um movimento possível aleatório
                current_game.move(random_move, current_game.turn)
                #print(f"Jogo atual{current_game}")
            else:
                break  # Se não houver movimentos possíveis, interrompe a simulação

        if current_game.check_winner():
            if current_game.player_winner == "X" and self.jogador == "O":
                return 0  # Vitória para X com o O como jogador
            elif current_game.player_winner == "O" and self.jogador == "O":
                return 1  # Vitória para O com o O como jogador
            elif current_game.player_winner == "X" and self.jogador == "X":
                return 1  # Vitória para o X com o X como jogador
            elif current_game.player_winner == "O" and self.jogador == "X":
                return 0  # Vitória para o O com o X como jogador
        else:
            return 0.5  # Empate
    
    def backpropagate(self, result):
        self.visits += 1
        self.wins += result
        if self.parent:  #  Chegou à raiz, não precisa de propagar mais para cima
            self.parent.backpropagate(result)
        
    def __str__(self):
        node_str = f"Game state: {self.game}\n"
        node_str += f"Wins: {self.wins}, Visits: {self.visits}\n"
        #node_str += f"Percentage: {node.ucb1_value()}"
        return node_str
        

In [115]:
from IPython.display import clear_output

def player_player(game):
    print(game)
    while not (game.winner or game.is_board_full()):
        
        if game.turn=="X":
            n=int(input(f"Player {game.turn}  turn: "))          #QUEM COMEÇA é o X 
            #score=game.get_score()
        else:
            n=int(input(f"Player {game.turn} turn: "))
            
        if not game.move(n, game.turn):
            print("Impossible move! Try again.")
        else:
            game.check_winner()
            print (game)
            #score=game.get_score()
            #print(f"Pontuação: {score}")
        
    
    if game.winner:
        print("Game Over! Player", game.player_winner, "wins!!")
        choice = input("Do you wanna play again? (y/n) ")
        if choice=="y":
            clear_output()
            menu_inicial()
    else:
        print("Game Over! It's a draw")
        choice = input("Do you wanna play again? (y/n) ")
        if choice=="y":
            clear_output()
            menu_inicial()
            
def player_algorithm(game,algorithm,h,jogador):
    print(game)
    
    while not (game.winner or game.is_board_full()):
        
        if jogador == True:
            n=int(input(f"Player {game.turn}  turn: "))          #QUEM COMEÇA é o X 
            if not game.move(n, game.turn):
                print("Impossible move! Try again.")
            else:
                game.check_winner()
                print(game)
            jogador = False
        else:
            if algorithm == '1':             #A Star
                if not game.check_winner():
                    #for j in range(len(sucessoreslist)):
                        #print(sucessoreslist[j])
                    _,prox_move=game.Astar(h)
                    #print(minimo)
                    #print(prox_move)
                    game.move(prox_move,game.turn)
                    game.check_winner()
                print(game)
            
            elif algorithm == '2':            # Monte carlo
                if not game.check_winner():
                    root_node = Node(game,game.turn)  # Criar o nó raiz para o Monte Carlo
                    game.monte_carlo(root_node, h)  # Realizar 1000 simulações de Monte Carlo
                    if root_node.expanded():
                        count=0
                        for child in root_node.children:
                            print(f"Vitórias do filho {count}: {child.wins} ")
                            print(f"Visitas do filho {count}: {child.visits} ")
                            print(f"Pontuação do filho: {root_node.ucb1_value(child)}")
                            count+=1
                        best_move = root_node.select_child().game.last_move
                        game.move(best_move, game.turn)
                        game.check_winner()
                    else:
                        print("Nenhum filho disponível para seleção.")

                print(game)
                    
            
            elif algorithm == '3':             #Minimax
                
                if not game.check_winner():
        
                    _,prox_move=game.minimax(5,h)
            
                    game.move(prox_move,game.turn)
                
                    game.check_winner()
                
                print(game)
                
            jogador = True
        
    
    if game.winner:
        print("Game Over! Player", game.player_winner, "wins!!")
        choice = input("Do you wanna play again? (y/n) ")
        if choice=="y":
            clear_output()
            menu_inicial()
    else:
        print("Game Over! It's a draw")
        choice = input("Do you wanna play again? (y/n) ")
        if choice=="y":
            clear_output()
            menu_inicial()

def algorithm_algorithm(game,algorithm_X,algorithm_O,h_X,h_O,testes):
    if testes == 1:
        print(game)
    
    
    while not (game.winner or game.is_board_full()):
    
        if game.turn=="X":
            if algorithm_X == '1':
                if not game.check_winner():
                    game.successors()
                    _,prox_move=game.Astar(h_X)
                    game.move(prox_move,game.turn)
                    game.check_winner()
                if testes == 1:
                    print(game)
            
            if algorithm_X == '2':
                if not game.check_winner():
                    root_node = Node(game,game.turn)  # Criar o nó raiz para o Monte Carlo
                    game.monte_carlo(root_node, h_X)  # Realizar 1000 simulações de Monte Carlo
                    for child in root_node.children:
                        print(f"Pontuação do filho: {root_node.ucb1_value(child)}")
                    
                    best_move = root_node.select_child().game.last_move
                    game.move(best_move, game.turn)
                    game.check_winner()
                if testes == 1: 
                    print(game)
            
            if algorithm_X == '3':
                if not game.check_winner():
                    _,prox_move=game.minimax(3,h_X)
                    game.move(prox_move,game.turn)
                    game.check_winner()
                if testes == 1:
                    print(game)
            
        if game.turn=="O":
            if algorithm_O == '1':
                if not game.check_winner():
                    game.successors()
                    _,prox_move=game.Astar(h_O)
                    game.move(prox_move,game.turn)
                    game.check_winner()
                if testes == 1:
                    print(game)
            
            if algorithm_O == '2':
                if not game.check_winner():
                    root_node = Node(game,game.turn)  # Criar o nó raiz para o Monte Carlo
                    game.monte_carlo(root_node, h_O)  # Realizar 1000 simulações de Monte Carlo
                    for child in root_node.children:
                        print(f"Pontuação do filho: {root_node.ucb1_value(child)}")
                    
                    best_move = root_node.select_child().game.last_move
                    game.move(best_move, game.turn)
                    game.check_winner()
                if testes == 1: 
                    print(game)
            
            if algorithm_O == '3':
                if not game.check_winner():
                    _,prox_move=game.minimax(3,h_O)
                    game.move(prox_move,game.turn)
                    game.check_winner()
                if testes == 1:
                    print(game)
    if testes == 1:    
        if game.winner:
            print("Game Over! Player", game.player_winner, "wins!!")
            choice = input("Do you wanna play again? (y/n) ")
            if choice=="y":
                clear_output()
                menu_inicial()
        else:
            print("Game Over! It's a draw")
            choice = input("Do you wanna play again? (y/n) ")
            if choice=="y":
                clear_output()
                menu_inicial()
    else:
        if game.winner:
            print(game)
            print(f"Venceu o {game.player_winner}")
            print("\n")
            print(f"Venha a próxima")
        else:
            print(game)
            print(f"Empate seus burros")

# Menu no Terminal

Depois poderá ser assim a escolha do modo, mas sendo isto uma versão bastante básica e simples

In [116]:
def menu_inicial():
    print("Welcome to Connect Four!")
    print("Choose your mode")
    print("1. Player vs Player")
    print("2. Player vs Algorithm")
    print("3. Algorithm vs Algorithm")
    print("4. Quit")
    
    choice = input("Enter your choice: ")
    
    if choice == '1':
        game = ConnectFour()
        player_player(game)
        
    elif choice == '2':
        clear_output()
        algorithm = menu_algoritmos()
        h = 0
        
        if algorithm == '1':
            clear_output()
            h = menu_dificuldade()
            #print(h)
            clear_output()
            comeca = menu_jogador()

            if comeca == '1':
                jogador = True
            else:
                jogador = False

            game = ConnectFour()
            player_algorithm(game,algorithm,h,jogador)
        
        elif algorithm == '2':
            clear_output()
            iteracoes = menu_iteracoes()
            clear_output()
            comeca = menu_jogador()

            if comeca == '1':
                jogador = True
            else:
                jogador = False

            game = ConnectFour()
            player_algorithm(game,algorithm,iteracoes,jogador)
        
        elif algorithm == '3':
            clear_output()
            h = menu_dificuldade()
            clear_output()
            comeca = menu_jogador()

            if comeca == '1':
                jogador = True
            else:
                jogador = False

            game = ConnectFour()
            player_algorithm(game,algorithm,h,jogador)
        
        elif algorithm == '4':
            clear_output()
            
            clear_output()
            comeca = menu_jogador()

            if comeca == '1':
                jogador = True
            else:
                jogador = False

            
            game = ConnectFour()
            player_algorithm(game,algorithm,h,jogador)
    
    elif choice == '3':
        
        count_X = 0
        count_O = 0
        count_draw = 0
        
        clear_output()
        testes = menu_testes()
        clear_output()
        algorithm_X, h_X = menu_algoritmo_algoritmo("X")
        clear_output()
        algorithm_O, h_O = menu_algoritmo_algoritmo("O")
        clear_output()
        #print(f"Dificuldade de X: {h_X}")
        #print(f"Dificuldade de O: {h_O}")
        
        for i in range (testes):
            game = ConnectFour()
            algorithm_algorithm(game, algorithm_X, algorithm_O, h_X, h_O, testes)
            if game.player_winner == "X":
                count_X += 1
            elif game.player_winner == "O":
                count_O += 1
            else:
                count_draw += 1
                
        if testes != 1:
            print(f"Vitorias do jogador X: {count_X}")
            print(f"Vitorias do jogador O: {count_O}")
            print(f"Empates: {count_draw}")
        
    elif choice == '4':
        print("Exiting the game. Goodbye!")
        
def menu_algoritmos():
    print("Please choose the algorithm")
    print("1. A*")
    print("2. Monte carlo")
    print("3. Minimax")
    print("4. AlphaBeta")
    print("5. Back")
    
    algorithm = input("Enter your choice: ")
    
    if algorithm == '5':
        clear_output()
        menu_inicial()
    
    return algorithm
    
def menu_dificuldade():
    print("Please choose the difficulty")
    print("1. Easy")
    print("2. Medium")
    print("3. Hard")
    
    difficulty = input("Enter your choice: ")
    
    return difficulty

def menu_algoritmo_algoritmo(jogador):
    print(f"Please choose the player {jogador} algorithm")
    print("1. A*")
    print("2. Monte carlo")
    print("3. Minimax")
    print("4. AlphaBeta")
    print("5. Back")
    
    algorithm = input("Enter your choice: ")
    
    h=0
    
    if algorithm == '1' or algorithm == '3':
        clear_output()
        h = menu_dificuldade()
    
    if algorithm == '2':
        clear_output()
        h = menu_iteracoes()
    
    return algorithm, h

def menu_testes():

    testes = int(input("Enter the number of tests (min 1): "))
    
    return testes

def menu_profundidade():
    
    profundidade = int(input("Enter the depth (min 1): "))
    
    return profundidade

def menu_iteracoes():
    
    iteracoes = int(input("Enter the iterations (min 1): "))
    
    return iteracoes

def menu_jogador():
    
    print(f"Please choose the player to start")
    print("1. You")
    print("2. Computer")
    
    comeca = input("Enter your choice: ")
    
    return comeca
    
menu_inicial()

Please choose the player to start
1. You
2. Computer
Enter your choice: 1

0 1 2 3 4 5 6
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -

Player X  turn: 3

0 1 2 3 4 5 6
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- - - - - - -
- - - X - - -

Vitórias do filho 0: 453.5 
Visitas do filho 0: 1360 
Pontuação do filho: 0.4541367787363013
Vitórias do filho 1: 532.0 
Visitas do filho 1: 1559 
Pontuação do filho: 0.4539602201652737
Vitórias do filho 2: 2246.0 
Visitas do filho 2: 5676 
Pontuação do filho: 0.45477389350789443
Vitórias do filho 3: 2211.5 
Visitas do filho 3: 5604 
Pontuação do filho: 0.454079802885208
Vitórias do filho 4: 799.0 
Visitas do filho 4: 2222 
Pontuação do filho: 0.4539998622047928
Vitórias do filho 5: 886.5 
Visitas do filho 5: 2436 
Pontuação do filho: 0.4540877655684814
Vitórias do filho 6: 368.0 
Visitas do filho 6: 1142 
Pontuação do filho: 0.4539383924292587

0 1 2 3 4 5 6
- - - - - - -
- - - - - - -
- - - - - - 

KeyboardInterrupt: Interrupted by user